In [3]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [4]:
rating_data = pd.read_csv('rating.csv')
recipe_data = pd.read_csv('recipe.csv')

In [5]:
rating_data

,userid,recipeid,rating,timestamp
0,1,1,5.0,1260759144
1,1,2,5.0,1260759179
2,2,1,4.0,1260759185
3,3,1,5.0,1260759182
4,4,1,3.0,1260759205
5,5,3,2.9,1260759208


In [6]:
recipe_data

,recipeid,title,specification
0,1,라면,"뜨겁다, 인스턴트, 싸다, 면"
1,2,빨계떡,"뜨겁다, 맵다, 인스턴트, 싸다, 면"
2,3,볶음밥,"밥, 고소하다, 계란, 감칠맛"
3,4,김밥,"밥, 싸다, 건강식, 간편하다"
4,5,아이스크림,"달다, 차갑다, 우유, 유제품"


In [7]:
rating_data.drop('timestamp', axis = 1, inplace = True)
recipe_data.drop('specification', axis = 1, inplace = True)

In [8]:
user_recipe_data = pd.merge(rating_data, recipe_data, on = 'recipeid')
user_recipe_data.head()

,userid,recipeid,rating,title
0,1,1,5.0,라면
1,2,1,4.0,라면
2,3,1,5.0,라면
3,4,1,3.0,라면
4,1,2,5.0,빨계떡


In [9]:
user_recipe_rating = user_recipe_data.pivot_table('rating', index = 'userid', columns='title').fillna(0)
user_recipe_rating

title,라면,볶음밥,빨계떡
userid,,,
1,5.0,0.0,5.0
2,4.0,0.0,0.0
3,5.0,0.0,0.0
4,3.0,0.0,0.0
5,0.0,2.9,0.0


In [10]:
recipe_user_rating = user_recipe_rating.values.T

In [11]:
SVD = TruncatedSVD(n_components=3)
matrix = SVD.fit_transform(recipe_user_rating)
matrix.shape

(3, 3)

In [12]:
corr = np.corrcoef(matrix)

In [13]:
recipe_title = user_recipe_rating.columns
recipe_title_list = list(recipe_title)
coffey_hands = recipe_title_list.index("빨계떡")

In [14]:
# 비슷한 형태의 음식 종류 출력
corr_coffey_hands  = corr[coffey_hands]
list(recipe_title[(corr_coffey_hands >= 0.1)])[:10]

['라면', '빨계떡']

In [15]:
# 사용자 개인 추천
df_rating = pd.read_csv('rating.csv')
df_recipe = pd.read_csv('recipe.csv')

In [16]:
df_user_recipe_ratings = df_rating.pivot(
    index='userid',
    columns='recipeid',
    values='rating'
).fillna(0)

In [17]:
df_user_recipe_ratings.head()

recipeid,1,2,3
userid,,,
1,5.0,5.0,0.0
2,4.0,0.0,0.0
3,5.0,0.0,0.0
4,3.0,0.0,0.0
5,0.0,0.0,2.9


In [18]:
# matrix는 pivot_table 값을 numpy matrix로 만든 것 
matrix = df_user_recipe_ratings.values

# user_ratings_mean은 사용자의 평균 평점 
user_ratings_mean = np.mean(matrix, axis = 1)

# R_user_mean : 사용자-레시피에 대해 사용자 평균 평점을 뺀 것.
matrix_user_mean = matrix - user_ratings_mean.reshape(-1, 1)

In [19]:
U, sigma, Vt = svds(matrix_user_mean, k = 2)
print(U.shape)
print(sigma.shape)
print(Vt.shape)
sigma = np.diag(sigma)

(5, 2)
(2,)
(2, 3)


In [20]:
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [21]:
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = df_user_recipe_ratings.columns)
df_svd_preds.head()

recipeid,1,2,3
0,5.000000e+00,5.000000e+00,-1.332268e-15
1,4.000000e+00,2.220446e-16,-1.110223e-15
2,5.000000e+00,6.661338e-16,-6.661338e-16
3,3.000000e+00,2.220446e-16,-4.440892e-16
4,-1.332268e-15,-5.551115e-16,2.900000e+00


In [22]:
def recommend_movies(df_svd_preds, user_id, ori_recipe_df, ori_ratings_df, num_recommendations=3):
    
    user_row_number = user_id - 1 
    
    sorted_user_predictions = df_svd_preds.iloc[user_row_number].sort_values(ascending=False)
    

    user_data = ori_ratings_df[ori_ratings_df.userid == user_id]
    

    user_history = user_data.merge(ori_recipe_df, on = 'recipeid').sort_values(['rating'], ascending=False)
    

    recommendations = ori_recipe_df[~ori_recipe_df['recipeid'].isin(user_history['recipeid'])]

    recommendations = recommendations.merge( pd.DataFrame(sorted_user_predictions).reset_index(), on = 'recipeid')

    recommendations = recommendations.rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
                      

    return user_history, recommendations

In [23]:
already_rated, predictions = recommend_movies(df_svd_preds, 2, df_recipe, df_rating, 3)
predictions

,recipeid,title,specification,Predictions
0,2,빨계떡,"뜨겁다, 맵다, 인스턴트, 싸다, 면",2.220446e-16
1,3,볶음밥,"밥, 고소하다, 계란, 감칠맛",-1.110223e-15
